# Calculating the Flux

Let:
- $\lambda$ be the wavelength in $\mu m$,
- $r$ be the radius of the telescope in $cm$,
- $E_{\text{ph}}$ be the energy per photon in $erg$,
- $F$ be the flux in $ph/s/DIT$ where $DIT$ is the detector integration time in $s$,
- $r$ be the telescope radius in $cm$,
- $c$ be the speed of light in $m/s$,
- $h$ be Planck's constant in $J \cdot s$,
- $R_{\text{vis}}$ be the resolution,
- $e_{\text{vis}}$ be the ?? in %
- $g_{\text{vis}}$ be the global throughput in %,
- $m_s$ be the source magnitude in $AB mag$,
- $m_0$ be the zero-point in $AB mag$.

$$\lambda_m = \lambda \times 10^{-6}$$

$$E_{\text{ph}} = \frac{c \times h}{\lambda_m} \times 10^{7}$$

$$F = \frac{10^{-0.4 \times (m_s - m_0)}}{E_{\text{ph}}} \times \frac{c}{\lambda_m^2} \times \pi r^2 \times \frac{\lambda_m}{R_{\text{vis}}} \times e_{\text{vis}} \times g_{\text{vis}}$$

# Calculating the background noise flux

Let:
- $N_{\text{sky}}$ be the background sky noise in $ph/s/m^2/\mu m/arcsec^2$,
- $r_{\text{elt}}$ be the radius of the telescope in $m$,
- $r_{\text{ap}}$ be the radius of the aperture in $m$.

$$F_{\text{bg}} = N_{\text{sky}} \times \pi r_{\text{elt}}^2 \times \frac{\lambda}{R_{\text{vis}}} \times \pi r_{\text{ap}}^2 \times g_{\text{vis}}$$

# Calculating the Signal to Noise Ratio

Let:
- $C$ be the signal counts in $ph/DIT$,
- $C_{\text{bg}}$ be the background counts in $ph/DIT$,
- $\text{DIT}$ be the detector integration time in $s$,
- $\text{NDIT}$ be the number of detector integrations,
- $\text{RON}_{\text{vis}}$ be the readout noise in $ph$,
- $\text{DARK}_{\text{vis}}$ be the dark current in $e/s/pixel$,
- $p_{\text{vis}}$ be the number of pixels,

$$C = F \times \text{DIT}$$

$$C_{\text{bg}} = F_{\text{bg}} \times \text{DIT}$$

$$SNR = \frac{C \times \sqrt{\text{NDIT}}}{\sqrt{C + C_{\text{bg}} + \text{RON}_{\text{vis}}^2 \times p_{\text{vis}} + \text{DARK}_{\text{vis}} \times \text{DIT} \times p_{\text{vis}}}}$$

In [7]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# If we're in Colab, we need to download the data folder from GitHub
if IN_COLAB:
    import os
    
    # Check if the data directory already exists or not
    if not 'data' in os.listdir():
        print("Downloading data...")
        os.makedirs("data", exist_ok=True)

        repository = "https://github.com/italoseara/MOSAIC-Prototype"
        files = ["data/Background.csv", "data/EE.csv", "data/Throughput.csv"]

        for file in files:
            os.system(f"wget {repository}/raw/main/{file} -O {file}")

DATA_DIR = "./data" if IN_COLAB else "../data"

In [8]:
import pandas as pd
from math import sqrt, pi

In [9]:
# Classes that will be used throughout the code

class LookupTable:
    """A class to represent a lookup table for retrieving data from a CSV file."""

    _df: pd.DataFrame
    _col: str
    
    def __init__(self, csv_file: str, col: str, **kwargs) -> None:
        self._df = pd.read_csv(csv_file, **kwargs)
        self._col = col

    def get(self, key: float) -> dict[str, float]:
        # NOTE: The excel file only gets the closest value that is smaller than the input.
        # Although I believe the other way would be more appropriate, I will
        # keep it as it is for now, to replicate the original behavior.

        smaller = self._df[self._df[self._col] <= key]
        closest_index = smaller[self._col].sub(key).abs().idxmin()
        # closest_index = self._df[self._col].sub(key).abs().idxmin()
        
        return self._df.loc[closest_index].to_dict()

In [10]:
# Constants
VELOCITY_OF_LIGHT = 299792458 # m/s
PLANCK_CONSTANT = 6.63e-34 # J.s
ZERO_POINT = -48.6 # erg/s/cm^2/Hz

# Telescope
ELT_DIAMETER = 38.542 # m

# Instrument
RESOLUTION_VIS = 5000
N_PIXELS_VIS = 175
DARK_CURRENT_VIS = 10 / 3600 # e/s/pixel
RON_VIS = 3 # e/pixel
APERTURE_DIAMETER = 0.7 # arcsec

In [11]:
# Inputs

ndit = int(input("Enter the number of exposures: "))
dit = float(input("Enter the exposure time (s): "))
wavelength = float(input("Enter the wavelength (um): "))
source = float(input("Enter the source magnitude: "))
sky = input("Enter the sky condition (Sky ALL, Sky NO MOON, Sky ALL+Th, Sky NO MOON+Th): ")

In [12]:
# NOTE: The background noise is calculated via other columns in the CSV file, and should not be used as a lookup table. For simplicity, I will use the lookup table to get the values.
gt_table = LookupTable(csv_file=f"{DATA_DIR}/Throughput.csv", col="Wavelength LR")
ee_table = LookupTable(csv_file=f"{DATA_DIR}/EE.csv", col="Wavelength") # EE = Encircled Energy

gt_vis = gt_table.get(wavelength)["VIS-LR"] # %
ee_vis = ee_table.get(wavelength * 1000)["EE-HR-ZA45 Durham VIS"] # %

print(f"Global Throughput: {gt_vis}")
print(f"EE: {ee_vis}")

# NOTE: Same thing as above, but for the sky noise.
bg_table = LookupTable(csv_file=f"{DATA_DIR}/Background.csv", col="Wavelength")

sky_noise = bg_table.get(wavelength * 1000)[sky] # ph/s/m^2/um/arcsec^2
print(f"Sky noise: {sky_noise}")

Global Throughput: 0.2133859479030071
EE: 0.5511021611116786
Sky noise: 529.9697591510001


In [13]:
# Calculate the photon flux

wavelength_m = wavelength * 1e-6
elt_area = pi * (ELT_DIAMETER * 100 / 2)**2
energy_per_photon = (VELOCITY_OF_LIGHT * PLANCK_CONSTANT) / (wavelength_m) * 1e7

flux = (
    10**(-0.4 * (source - ZERO_POINT)) / energy_per_photon * 
    VELOCITY_OF_LIGHT / (wavelength_m)**2 * elt_area * 
    wavelength_m / RESOLUTION_VIS * 
    ee_vis * gt_vis
) # ph/s/DIT

counts = flux * dit # ph/DIT

print(f"Flux: {flux}")
print(f"Counts: {counts}")

Flux: 5.982363158868796
Counts: 1196.472631773759


In [14]:
# Calculate the background noise flux

elt_area = pi * (ELT_DIAMETER / 2)**2 # m^2
aperture_area = pi * (APERTURE_DIAMETER / 2)**2 # arcsec^2

bg_flux = sky_noise * elt_area * (wavelength / RESOLUTION_VIS) * aperture_area * gt_vis # ph/s/DIT
bg_counts = bg_flux * dit # ph/DIT

print(f"Background flux: {bg_flux}")
print(f"Background counts: {bg_counts}")

Background flux: 6.093158678942489
Background counts: 1218.6317357884977


In [15]:
# Calculate the SNR

snr = counts * sqrt(ndit) / sqrt(counts + bg_counts + RON_VIS**2 * N_PIXELS_VIS + DARK_CURRENT_VIS * dit * N_PIXELS_VIS)

print(f"S/R: {snr}")

S/R: 132.33298256021814
